In [1]:
!pip install mlflow
import mlflow

In [4]:
%%bash
export DATA_PATH=./data
wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet --directory-prefix $DATA_PATH
wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet --directory-prefix $DATA_PATH
wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet --directory-prefix $DATA_PATH
python3 ./02-experiment-tracking/lib/preprocess_data.py --raw_data_path $DATA_PATH --dest_path ./output
python3 ./02-experiment-tracking/lib/train.py
echo Run "mlflow ui --backend-store-uri sqlite:///mlflow.db"  # Set in train.py line 10


File ‘./data/green_tripdata_2023-01.parquet’ already there; not retrieving.

File ‘./data/green_tripdata_2023-02.parquet’ already there; not retrieving.

File ‘./data/green_tripdata_2023-03.parquet’ already there; not retrieving.

2024/09/27 11:58:27 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/27 11:58:28 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Run mlflow ui --backend-store-uri sqlite:///mlflow.db


Run for Q3: `mlflow ui --backend-store-uri sqlite:///mlflow.db`

Run for Q4: `mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts`


In [12]:
%%bash
pip install hyperopt
python3 ./02-experiment-tracking/lib/hpo.py

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/27 12:11:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run tasteful-loon-649 at: http://127.0.0.1:5000/#/experiments/2/runs/2c44f9182e49436f850455a96d47cae0.

2024/09/27 12:11:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2.

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/27 12:11:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run placid-ram-495 at: http://127.0.0.1:5000/#/experiments/2/runs/614cbd124fee4c9397a58db2a0aa6f0

100%|██████████| 15/15 [01:08<00:00,  4.55s/trial, best loss: 5.335419588556921]


In [24]:
%%bash
python3 ./02-experiment-tracking/lib/register_model.py

2024/09/27 12:20:48 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/09/27 12:20:48 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/27 12:20:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecat

In [26]:
import os

# Q1. Install MLflow
print(f"Q1. What's the version that you have?\n  Answer: {mlflow.__version__}")  # 2.16.2

# Q2. Download and preprocess the data
print(f"Q2. How many files were saved to OUTPUT_FOLDER?\n  Answer: {os.popen('ls ./output | wc -w').read().strip()}")  # 4

# Q3. Train a model with autolog
mlflow.set_tracking_uri("sqlite:///mlflow.db")
experiment_id = mlflow.search_experiments(filter_string="name = 'random-forest-train'")[0].experiment_id
run_id = mlflow.search_runs(experiment_ids=experiment_id).iloc[0]["run_id"]
print(f"Q3. What is the value of the min_samples_split parameter?\n  Answer: {mlflow.get_run(run_id=run_id).data.params['min_samples_split']}")  # 2

# Q4. Launch the tracking server locally
print("Q4. In addition to backend-store-uri, what else do you need to pass to properly configure the server?\n  Answer: default-artifact-root")  # default-artifact-root

# Q5. Tune model hyperparameters
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = mlflow.search_experiments(filter_string="name = 'random-forest-hyperopt'")[0].experiment_id
run_id = mlflow.search_runs(experiment_ids=experiment_id, order_by=["metrics.rmse ASC"]).iloc[0]["run_id"]
rmse = mlflow.get_run(run_id=run_id).data.metrics["rmse"]
print(f"Q5. What's the best validation RMSE that you got?\n  Answer: {rmse:0.3f}")  # 5.335

# Q6. Promote the best model to the model registry
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = mlflow.search_experiments(filter_string="name = 'random-forest-best-models'")[0].experiment_id
run_id = mlflow.search_runs(experiment_ids=experiment_id, order_by=["metrics.test_rmse ASC"]).iloc[0]["run_id"]
test_rmse = mlflow.get_run(run_id=run_id).data.metrics["test_rmse"]
print(f"Q6. What is the test RMSE of the best model?\n  Answer: {test_rmse:0.3f}")  # 5.567


Q1. What's the version that you have?
  Answer: 2.16.2
Q2. How many files were saved to OUTPUT_FOLDER?
  Answer: 4
Q3. What is the value of the min_samples_split parameter?
  Answer: 2
Q4. In addition to backend-store-uri, what else do you need to pass to properly configure the server?
  Answer: default-artifact-root
Q5. What's the best validation RMSE that you got?
  Answer: 5.335
Q6. What is the test RMSE of the best model?
  Answer: 5.567
